# Imports

In [1]:
import numpy as np
from tensorflow.keras.utils import set_random_seed
import os

# assigning random seed for reproducebility was taken from: https://stackoverflow.com/questions/51249811/reproducible-results-in-tensorflow-with-tf-set-random-seed
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
np.random.seed(seed)
set_random_seed(seed)


import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from keras.applications import ConvNeXtTiny

___________________________________________________________________________________________________________________________________________________________________________________________________
# Neural Network

Here we build a basic neural network to classify out images. The below code repurposed many aspects of previous projects I worked on with colleagues. Most of the neural network and metrics takes inspiration from [A plant disease classification project](https://github.com/DerikVo/DSI_project_4_plant_disease) and a single day [Hack-a-thon](https://github.com/DerikVo/NN_hackathon) to classify if an object was a hotdog or not a hotdog.

In [2]:
# set the training and testing paths
training_folder_path = '../Images/Training'
testing_folder_path = '../Images/Testing'

In [3]:
# manually list out the class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

In [4]:
datagen = ImageDataGenerator(validation_split=0.30)
# Get the training data
train_ds = datagen.flow_from_directory(
    training_folder_path,
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    classes=class_names,
    class_mode='categorical',
    subset='training',  # Set as training data
    seed=42
)

# Get the validation data
val_ds = datagen.flow_from_directory(
    training_folder_path,
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    classes=class_names,
    class_mode='categorical',
    subset='validation',  # Set as validation data
    seed=42,
    shuffle=False
)

# Get the test data
test_ds = datagen.flow_from_directory(
    testing_folder_path,
    target_size=(256, 256),
    color_mode='grayscale',
    class_mode='categorical',
    seed=42,
    shuffle=False
)


Found 4000 images belonging to 4 classes.
Found 1712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


## First model
This model uses many aspects of a prior project for [plant disease classification](https://github.com/DerikVo/DSI_project_4_plant_disease/tree/main)

In [5]:
early_stopping = EarlyStopping(patience=5)

In [6]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[early_stopping])

Epoch 1/10
125/125 [==============================] - 87s 574ms/step - loss: 6.8223 - accuracy: 0.7800 - val_loss: 1.1457 - val_accuracy: 0.7775
Epoch 2/10
125/125 [==============================] - 50s 402ms/step - loss: 0.2138 - accuracy: 0.9258 - val_loss: 1.0864 - val_accuracy: 0.7079
Epoch 3/10
125/125 [==============================] - 54s 432ms/step - loss: 0.1285 - accuracy: 0.9575 - val_loss: 1.2109 - val_accuracy: 0.7623
Epoch 4/10
125/125 [==============================] - 81s 649ms/step - loss: 0.0572 - accuracy: 0.9820 - val_loss: 1.7071 - val_accuracy: 0.7407
Epoch 5/10
125/125 [==============================] - 55s 441ms/step - loss: 0.0271 - accuracy: 0.9905 - val_loss: 1.7876 - val_accuracy: 0.7815
Epoch 6/10
125/125 [==============================] - 82s 654ms/step - loss: 0.0348 - accuracy: 0.9902 - val_loss: 1.7913 - val_accuracy: 0.7523
Epoch 7/10
125/125 [==============================] - 81s 649ms/step - loss: 0.0084 - accuracy: 0.9985 - val_loss: 1.7633 - val_ac

In [9]:
model.save('../Models/CNN_base.h5')

### Interpretation:
Here we see that our training accuracy is about 99% while our validation is 76% which suggest our model is very overfit. We will need to either reduce features or add some regularization. The validation score is higher than our baseline, but the score is lower than [Munir)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7794124/) and their team's study accuracy of 87% (N=154). However, this is simply a supportive tool to assist radiologist, and the radiologist response would continue to train the model.

For out next iteration, lets try adding some regularization to see if it can reduce overfitting so our model can be more generalized.

## Second model
This model uses regularization to try to combat overfiting. The model uses techniques learned from the [General Assembly Data science immersive bootcamp](https://generalassemb.ly/education/data-science) which taught a lab on regularization with convolutional neural networks.

In [10]:
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1), kernel_regularizer=l2(0.01)))
model2.add(MaxPooling2D((2, 2)))

model2.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model2.add(Dropout(0.5))

model2.add(Dense(4, activation='softmax'))

In [11]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
history2 = model2.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[early_stopping])

Epoch 1/10
125/125 [==============================] - 60s 458ms/step - loss: 21.2878 - accuracy: 0.6155 - val_loss: 6.0345 - val_accuracy: 0.6232
Epoch 2/10
125/125 [==============================] - 56s 449ms/step - loss: 4.9348 - accuracy: 0.7905 - val_loss: 5.2085 - val_accuracy: 0.5917
Epoch 3/10
125/125 [==============================] - 86s 683ms/step - loss: 3.9861 - accuracy: 0.8510 - val_loss: 4.1504 - val_accuracy: 0.7009
Epoch 4/10
125/125 [==============================] - 55s 441ms/step - loss: 3.2915 - accuracy: 0.8687 - val_loss: 3.8028 - val_accuracy: 0.7062
Epoch 5/10
125/125 [==============================] - 58s 460ms/step - loss: 2.8297 - accuracy: 0.8915 - val_loss: 3.3022 - val_accuracy: 0.7447
Epoch 6/10
125/125 [==============================] - 85s 678ms/step - loss: 2.4836 - accuracy: 0.8950 - val_loss: 2.9780 - val_accuracy: 0.7436
Epoch 7/10
125/125 [==============================] - 56s 445ms/step - loss: 2.2336 - accuracy: 0.9015 - val_loss: 2.7743 - val_a

In [13]:
model2.save('../Models/CNN_regularization.h5')

### Interpretation:
Here we see that our training accuracy is about 90% while our validation is 73% which suggest our model is still overfit, but not as much. The model does better than our baseline model's accuracy of 46% but is less than the accuracy in [Munir's team's](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7794124/) study which found the accuracy of two radiologist was 87%.

Since our validation score was higher without regularization we will stick with the first model. 

In [14]:
import sys
sys.path.append('../modules/')
import model as m
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

__________________________________________________________________________________________________________________________________________________________________________________________________________________
# Conclusion:

It appears our neural networks have similar scores. These score better than our baseline of 46%, but does less than the accuracy (87%) of the radiologist found in the [Munir et al. (2021)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7794124/) study. It should noted their sample size was 154 patients while this data set had over 7000 images; however, we need to keep in mind multiple images could be of the same patient.

A neural network implementing augmentation was attempted, but that was an issue with running out of memory. There was attempts at saving the images instead, but that was causing conflicts as well so augmentation was scrapped.

We will now proceed to our [Modeling Evaluation Notebook](../Notebooks/04_Model_evaluation.ipynb) to evaluate our models even further.